範例目標:<br>
1. 實做欄位索引之間轉換
2. 重新組織資料

範例重點:<br>
1. 不管是欄位轉索引或是索引轉欄位，皆由最外層的開始轉換
2. 重新組織資料時應注意參數的理解，可以多做嘗試

In [1]:
import pandas as pd
import numpy as np

In [2]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
df = pd.DataFrame(data, index=index, columns=columns)
df

subject     Bob      Guido       Sue     
type         HR Temp    HR Temp   HR Temp
year visit                               
2013 1      1.2 -0.3  -1.8 -1.8 -0.4 -0.1
     2      1.7 -0.4   2.2  0.3 -0.9 -0.9
2014 1     -0.7  1.3   0.5  0.3  0.1 -1.0
     2      0.3 -1.5   0.2 -1.1  1.6 -0.7

In [3]:
#欄位轉索引 : 將一欄位(column)轉成一索引(index)，使用.stack()即可，可以將type這個欄位轉成了索引，所以索引變成了year、visit、type
df.stack()

subject          Bob  Guido  Sue
year visit type                 
2013 1     HR    1.2   -1.8 -0.4
           Temp -0.3   -1.8 -0.1
     2     HR    1.7    2.2 -0.9
           Temp -0.4    0.3 -0.9
2014 1     HR   -0.7    0.5  0.1
           Temp  1.3    0.3 -1.0
     2     HR    0.3    0.2  1.6
           Temp -1.5   -1.1 -0.7

In [4]:
#再做一次.stack()索引變成了year、visit、type、subject
df.stack().stack()

year  visit  type  subject
2013  1      HR    Bob        1.2
                   Guido     -1.8
                   Sue       -0.4
             Temp  Bob       -0.3
                   Guido     -1.8
                   Sue       -0.1
      2      HR    Bob        1.7
                   Guido      2.2
                   Sue       -0.9
             Temp  Bob       -0.4
                   Guido      0.3
                   Sue       -0.9
2014  1      HR    Bob       -0.7
                   Guido      0.5
                   Sue        0.1
             Temp  Bob        1.3
                   Guido      0.3
                   Sue       -1.0
      2      HR    Bob        0.3
                   Guido      0.2
                   Sue        1.6
             Temp  Bob       -1.5
                   Guido     -1.1
                   Sue       -0.7
dtype: float64

In [5]:
#索引轉欄位: 將一索引(index)轉成一欄位(column) ，使用.unstack()即可，可以將visit這個索引轉成了欄位，所以欄位變成了subject、type 、visit
df.unstack()

subject  Bob                Guido                 Sue               
type      HR      Temp         HR      Temp        HR      Temp     
visit      1    2    1    2     1    2    1    2    1    2    1    2
year                                                                
2013     1.2  1.7 -0.3 -0.4  -1.8  2.2 -1.8  0.3 -0.4 -0.9 -0.1 -0.9
2014    -0.7  0.3  1.3 -1.5   0.5  0.2  0.3 -1.1  0.1  1.6 -1.0 -0.7

In [6]:
df = pd.DataFrame({'Name':{0:'John', 1:'Bob', 2:'Shiela'}, 
                   'Course':{0:'Masters', 1:'Graduate', 2:'Graduate'}, 
                   'Age':{0:27, 1:23, 2:21}}) 
df

,Name,Course,Age
0,John,Masters,27
1,Bob,Graduate,23
2,Shiela,Graduate,21


In [7]:
#全部轉成欄位（寬表轉長表）
df.melt()

,variable,value
0,Name,John
1,Name,Bob
2,Name,Shiela
3,Course,Masters
4,Course,Graduate
5,Course,Graduate
6,Age,27
7,Age,23
8,Age,21


In [8]:
#保留Name欄位其餘轉成欄位值
df.melt(id_vars='Name')

,Name,variable,value
0,John,Course,Masters
1,Bob,Course,Graduate
2,Shiela,Course,Graduate
3,John,Age,27
4,Bob,Age,23
5,Shiela,Age,21


In [9]:
#保留Name欄位其餘轉成欄位值，之後再留下value_vars='Name'
df.melt(value_vars='Name')

,variable,value
0,Name,John
1,Name,Bob
2,Name,Shiela


In [10]:
df = pd.DataFrame({'fff': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bbb': ['P', 'Q', 'R', 'P', 'Q', 'R'],
                   'baa': [2, 3, 4, 5, 6, 7],
                   'zzz': ['h', 'i', 'j', 'k', 'l', 'm']})
df

,fff,bbb,baa,zzz
0,one,P,2,h
1,one,Q,3,i
2,one,R,4,j
3,two,P,5,k
4,two,Q,6,l
5,two,R,7,m


In [11]:
#.pivot()函數根據給定的索引/列值重新組織給定的DataFrame
#參數
#index : 新資料的索引名稱
#columns: 新資料的欄位名稱
#values :新資料的值名稱

df.pivot(index='fff', columns='bbb', values='baa')

bbb,P,Q,R
fff,,,
one,2,3,4
two,5,6,7


In [12]:
# pivot_table方法實現了類似pivot方法的功能，它可以在指定的列和行有重複的情況下使用，我們可以使用均值、中值或其他的聚合函式來計算重複條目中的單個值。
# 參數

# index : 新資料的索引名稱
# columns : 新資料的欄位名稱
# values : 新資料的值名稱
# aggfunc : 重複數字的函數邏輯(均值、中值或其他的聚合函式)

table = pd.DataFrame({'Item': ['Item0','Item0','Item0','Item1'],
             'CType': ['Gold','Bronze','Gold','Silver'],
             'USD': [1,2,3,4],
             'EU': [1.1,2.2,3.3,4.4]})

table

,Item,CType,USD,EU
0,Item0,Gold,1,1.1
1,Item0,Bronze,2,2.2
2,Item0,Gold,3,3.3
3,Item1,Silver,4,4.4


In [13]:
#可以發現左表格Item、CType欄位，有兩個重複的值(Item0,Gold)在ix=0,1兩列，因為aggfunc=np.mean，所以針對這兩筆資料做平均
p=table.pivot_table(index='Item',columns='CType',values='USD', aggfunc=np.mean)
p

CType,Bronze,Gold,Silver
Item,,,
Item0,2.0,2.0,NaN
Item1,NaN,NaN,4.0
